# Checking and describing the generated data

It is always beneficial to add a notebook that quickly looks into the data to help you remember, which data you collected and if it actually looks correct.

In [86]:
from algbench import describe, read_as_pandas, Benchmark
from _conf import EXPERIMENT_DATA

In [87]:
describe(EXPERIMENT_DATA)

An entry in the database can look like this:
_____________________________________________
 result:
| num_nodes: 225
| lower_bound: 123306.0
| objective: 127043.0
 timestamp: 2023-11-18T11:19:19.123580
 runtime: 90.27978515625
 stdout: [[0.0006453990936279297, 'Set parameter Username\n'], [0.0014081001281738281,...
 stderr: []
 logging: [{'name': 'Evaluation', 'msg': 'Building model.', 'args': [], 'levelname': 'I...
 env_fingerprint: f685530fc54b6589eacf1065fce3d62b786d9f2c
 args_fingerprint: 330cdf828a4d11ea695f982a134e7d346d324025
 parameters:
| func: run_solver
| args:
|| instance_name: ts225
|| time_limit: 90
|| strategy: GurobiTspSolver
|| opt_tol: 0.001
 argv: ['01_run_benchmark.py']
 env:
| hostname: workstation-r7
| python_version: 3.10.12 (main, Jul  5 2023, 18:54:27) [GCC 11.2.0]
| python: /home/krupke/anaconda3/envs/mo310/bin/python3
| cwd: /home/krupke/Repositories/cpsat-primer/examples/tsp_evaluation copy
| git_revision: c09ff89ad7fa790631c7ee819008e77caaf64c64
| python_fi

In [88]:
t = read_as_pandas(
    EXPERIMENT_DATA,
    lambda entry: {
        "instance_name": entry["parameters"]["args"]["instance_name"],
        "num_nodes": entry["result"]["num_nodes"],
        "time_limit": entry["parameters"]["args"]["time_limit"],
        "strategy": entry["parameters"]["args"]["strategy"],
        "opt_tol": entry["parameters"]["args"]["opt_tol"],
        "runtime": entry["runtime"],
        "objective": entry["result"]["objective"],
        "lower_bound": entry["result"]["lower_bound"],
    },
)
t.drop_duplicates(inplace=True, subset=["instance_name", "num_nodes", "strategy", "opt_tol"])
t["opt_gap"] = (t["objective"] - t["lower_bound"]) / t["lower_bound"]
t.sort_values(["num_nodes", "instance_name"])

,instance_name,num_nodes,time_limit,strategy,opt_tol,runtime,objective,lower_bound,opt_gap
85,gr48,0,90,GurobiTspSolver,0.001,0.000970,0.0,0.0,NaN
86,gr48,0,90,GurobiTspSolver,0.010,0.000754,0.0,0.0,NaN
87,gr48,0,90,GurobiTspSolver,0.050,0.000731,0.0,0.0,NaN
88,gr48,0,90,GurobiTspSolver,0.100,0.000726,0.0,0.0,NaN
89,gr48,0,90,GurobiTspSolver,0.250,0.000717,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...
15,ts225,225,90,CpSatTspSolverMtz,0.001,93.731544,149823.0,116621.0,0.284700
16,ts225,225,90,CpSatTspSolverMtz,0.010,96.672324,143348.0,116621.0,0.229178
17,ts225,225,90,CpSatTspSolverMtz,0.050,93.176805,180804.0,116930.0,0.546258
18,ts225,225,90,CpSatTspSolverMtz,0.100,93.305313,159004.0,117239.0,0.356238


In [89]:
for entry in Benchmark(EXPERIMENT_DATA):
    
    if entry["parameters"]["args"]["instance_name"] == "random_euclidean_100_1" and entry["parameters"]["args"]["strategy"] == "GurobiTspSolver":
        print("=====================================")
        stdout = "".join( e[1] for e in entry["stdout"])
        stderr = "".join( e[1] for e in entry["stderr"])
        print(stdout)
        print(stderr)
        if not stdout.strip():
            print("No stdout")
        print("=====================================")

## Check for errors in the data

You always want to check if the results you got are actually feasible. Errors easily happen and are not always visible on the plots.
Thus, you want to do some basic checks to detect errors early on. For example, you could accidentally have swapped lower and upper bounds in the data generation process.
Depending on your plots, this may not be visible, and you may end up comparing the wrong data and draw the wrong conclusions.
Or, you could have accidentally swapped runtime and objective values, which could look reasonable in the data as the runtime and the objective often increase with the instance size.

A very basic check is to check if the best lower and upper bounds do not contradict each other. Many errors will be caught by this check. However, you often need some tolerance to account for numerical errors.

In [90]:
assert (t.dropna()["opt_gap"]>=-0.0001).all(), "Optimality gap is negative!"

In [91]:
# Always make sure that your results are not trivially wrong
#  - e.g. lower bound is higher than objective
max_lb = t.groupby(["instance_name"])["lower_bound"].max()
min_obj = t.groupby(["instance_name"])["objective"].min()
eps = 0.0001  # some tolerance is needed when working with floats.
bad_instances = max_lb[max_lb-min_obj>eps*max_lb].index.to_list()
from IPython.display import display
display(t[t["instance_name"].isin(bad_instances)])
assert len(bad_instances) == 0, "Bad instances detected: {}".format(bad_instances)

,instance_name,num_nodes,time_limit,strategy,opt_tol,runtime,objective,lower_bound,opt_gap
